In [1]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

In [2]:
split = 2

In [3]:
adata_pred_ood = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_ood_with_predictions_{split}.h5ad")
adata_pred_test = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/otfm/combosciplex/adata_test_with_predictions_{split}.h5ad")

In [4]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [5]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [6]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [7]:
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="X_recon_pred")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].layers["X_recon_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="X_recon_pred")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].layers["X_recon_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

In [8]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [9]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [10]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [11]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [12]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [13]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9614297534177275,
 'decoded_ood_sinkhorn_div_1': 121.20053972516742,
 'decoded_ood_sinkhorn_div_10': 73.18851143973214,
 'decoded_ood_sinkhorn_div_100': 4.215419224330357,
 'decoded_ood_e_distance': 7.557114017899512,
 'decoded_ood_mmd': 0.012877070744122778}

In [14]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': 0.7229328559695293,
 'encoded_ood_sinkhorn_div_1': 6.05950791495187,
 'encoded_ood_sinkhorn_div_10': 3.925544193812779,
 'encoded_ood_sinkhorn_div_100': 3.229284014020647,
 'encoded_ood_e_distance': 6.296707337428846,
 'encoded_ood_mmd': 0.04003262393442648}

In [15]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9548519469553227,
 'deg_ood_sinkhorn_div_1': 14.538438524518694,
 'deg_ood_sinkhorn_div_10': 2.1465723855154857,
 'deg_ood_sinkhorn_div_100': 1.5328265598842077,
 'deg_ood_e_distance': 2.952364122127638,
 'deg_ood_mmd': 0.014639649673232011}

In [16]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.9915046221047469,
 'deg_test_sinkhorn_div_1': 12.223303576310476,
 'deg_test_sinkhorn_div_10': 1.1829545100529988,
 'deg_test_sinkhorn_div_100': 0.29660717646280926,
 'deg_test_e_distance': 0.48216651858217446,
 'deg_test_mmd': 0.011100951737413803}

In [17]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9867476304597527,
 'decoded_test_e_distance': 2.456588857716172,
 'decoded_test_mmd_distance': 0.013475001944849888}

In [18]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9614297534177275,
 'decoded_ood_sinkhorn_div_1': 121.20053972516742,
 'decoded_ood_sinkhorn_div_10': 73.18851143973214,
 'decoded_ood_sinkhorn_div_100': 4.215419224330357,
 'decoded_ood_e_distance': 7.557114017899512,
 'decoded_ood_mmd': 0.012877070744122778}

In [19]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9548519469553227,
 'deg_ood_sinkhorn_div_1': 14.538438524518694,
 'deg_ood_sinkhorn_div_10': 2.1465723855154857,
 'deg_ood_sinkhorn_div_100': 1.5328265598842077,
 'deg_ood_e_distance': 2.952364122127638,
 'deg_ood_mmd': 0.014639649673232011}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.9867476304597527,
 'decoded_test_e_distance': 2.456588857716172,
 'decoded_test_mmd_distance': 0.013475001944849888}

In [21]:
mean_test_metrics_encoded

{'encoded_test_r_squared': 0.9830164989165135,
 'encoded_test_sinkhorn_div_1': 3.8881096839904785,
 'encoded_test_sinkhorn_div_10': 0.957401712735494,
 'encoded_test_sinkhorn_div_100': 0.27971092859903973,
 'encoded_test_e_distance': 0.4167927232302316,
 'encoded_test_mmd': 0.013793751772027463}

In [22]:
ood_metrics_encoded

{'Dacinostat+Dasatinib': {'r_squared': 0.40132852594045065,
  'sinkhorn_div_1': 5.30843448638916,
  'sinkhorn_div_10': 3.087503433227539,
  'sinkhorn_div_100': 2.5542030334472656,
  'e_distance': 5.005057997199014,
  'mmd': 0.03596945},
 'Dacinostat+PCI-34051': {'r_squared': 0.5195711149017668,
  'sinkhorn_div_1': 4.748167037963867,
  'sinkhorn_div_10': 2.701266288757324,
  'sinkhorn_div_100': 2.1962738037109375,
  'e_distance': 4.287860070574196,
  'mmd': 0.033522416},
 'Givinostat+Cediranib': {'r_squared': 0.9821067810911009,
  'sinkhorn_div_1': 1.4607257843017578,
  'sinkhorn_div_10': 0.3954582214355469,
  'sinkhorn_div_100': 0.2600555419921875,
  'e_distance': 0.5030949148071358,
  'mmd': 0.005996241},
 'Givinostat+Curcumin': {'r_squared': 0.9681639899723552,
  'sinkhorn_div_1': 1.660163402557373,
  'sinkhorn_div_10': 0.6130685806274414,
  'sinkhorn_div_100': 0.4730186462402344,
  'e_distance': 0.9143473408152047,
  'mmd': 0.006597597},
 'Panobinostat+Alvespimycin': {'r_squared': 0

In [23]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/otfm"

In [24]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
